# Current Topics and Projects
## 2. Perspective toxicity score

In [1]:
'''
This file requests the perspective api scores and saves them to the data..
'''

import json
import requests 
import time
import pandas as pd
import csv
from tqdm import tqdm
import threading
from queue import Queue


In [2]:
# Read the CSV file and create a DataFrame
df = pd.read_csv('/kaggle/input/reddit-sample/all_comments_sample_137K.csv')
df

,id,body,subreddit,to_type,NEL,Names,created_utc,sex,ethnicity,origin,...,nickname_used,Adjectives,Verbs,Nouns,Descriptors_parsed,Verbs_parsed,Relation,Valence,Arousal,Dominance
0,e2r5x8o,"Paul Joseph Watson, Count Dankula and more ha...",The_Donald,t1,Q291169,['Jeremy Corbyn'],NaN,male,NaN,United Kingdom,...,False,"['more', 'parliamentary', 'small', 'effective'...","['have', 'be', 'have', 'be', 'take', 'get', 'g...","['system', 'party', 'system', 'seat', 'winner'...",[],[],['nmod'],0.570419,0.425279,0.563581
1,e2r5x9y,Just wait til [NAME] releases [NAME] tapes! .....,politics,t1,Q7747,['Putin'],NaN,male,Russians,Russia,...,False,[],['wait'],"['name', 'release', 'name', 'tape', 'music', '...",[],[],"['nmod', 'npadvmod']",0.569800,0.381400,0.425200
2,e2r5zwt,"I want Michael Avenatti to live a long time, a...",politics,t1,Q22686,['Donald Trump'],NaN,male,NaN,United States of America,...,False,['long'],"['want', 'live', 'want', 'lose', 'say', 'need'...","['time', 'dignity', 'name', 'name', 'poison', ...",[],['say'],['dep'],0.600769,0.472000,0.527231
3,e2r640r,I think we've established [NAME]'s 'type'...ty...,politics,t3,Q22686,['Trump'],NaN,male,NaN,United States of America,...,False,"['young', 'nubile']","['think', 'establish', 'prefer', 'remind']","['name', ""type'"", 'type', 'chick', 'type']",[],['establish'],['dobj'],0.669714,0.372143,0.507857
4,e2r66h0,There's no way [NAME] or Trump know exactly wh...,politics,t3,Q212648,['Giuliani'],NaN,male,NaN,United States of America,...,False,"['able', 'several', 'treasonous', 'mere', 'rus...","['know', 'record', 'be', 'get', 'expect', 'rel...","['way', 'name', 'trump', 'name', 'trump', 'mag...","['trump', 'able']",['be'],"['parataxis', 'dep']",0.549697,0.484636,0.545667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137550,fah1q9z,Reminds me of Spiro Agnew a little bit. Spiro ...,neoliberal,t3,Q9588,['Nixon'],NaN,male,NaN,United States of America,...,False,"['little', 'moderate', 'republican', 'more', '...","['remind', 'start', 'start', 'move', 'think', ...","['bit', 'governor', 'right', 'leader', 'exampl...","['immediately', 'speech']",[],"['dep', 'pobj']",0.150183,0.120585,0.151841
137551,fah1wk9,How would Warren’s plan to get the most vulner...,politics,t1,Q359442,['Bernie'],NaN,male,Jewish people,United States of America,...,True,"['vulnerable', 'unwinable', 'dangerous', 'real...","['get', 'cover', 'hurt', 'cover', 'think', '’'...","['plan', 'society', 'name', 'battle', 'year', ...",['plan'],"['think', 'pass']","['npadvmod', 'appos', 'dobj']",0.233210,0.193880,0.240870
137552,fah2hvs,This’s [NAME],news,t1,Q7791016,['Barrack'],NaN,male,NaN,United States of America,...,True,[],['’'],['name'],[],['’'],['appos'],0.114600,0.060000,0.083000
137553,fah2lbq,Mid year exam (the [NAME] email investigation)...,The_Donald,t3,Q6294,['Hillary'],NaN,female,NaN,United States of America,...,False,['mid'],[],"['year', 'exam', 'name', 'email', 'investigati...",['email'],[],['nmod'],0.237357,0.170286,0.217071


In [3]:
df.columns

Index(['id', 'body', 'subreddit', 'to_type', 'NEL', 'Names', 'created_utc',
       'sex', 'ethnicity', 'origin', 'DOB', 'highest_position', 'party',
       'entity_given_name', 'entity_family_name', 'given_name_used',
       'family_name_used', 'full_name_used', 'nickname_used', 'Adjectives',
       'Verbs', 'Nouns', 'Descriptors_parsed', 'Verbs_parsed', 'Relation',
       'Valence', 'Arousal', 'Dominance'],
      dtype='object')

In [4]:
api_key =""

In [5]:
class RateLimiter:
    def __init__(self, rate_per_second):
        self.rate_per_second = rate_per_second
        self.lock = threading.Lock()
        self.request_times = []

    def acquire(self):
        with self.lock:
            current_time = time.time()
            self.request_times = [t for t in self.request_times if current_time - t < 1]

            if len(self.request_times) >= self.rate_per_second:
                sleep_time = 1 - (current_time - self.request_times[0])
                time.sleep(sleep_time)
                self.request_times = self.request_times[1:]

            self.request_times.append(time.time())

rate_limiter = RateLimiter(50)

def get_perspective_scores(text):
    '''
    This function retrieves the api scores and enforces a rate limit.
    '''
    url = "https://commentanalyzer.googleapis.com/v1alpha1/comments:analyze"
    data = {
        "comment": {"text": text},
        "requestedAttributes": {
            "TOXICITY": {},
            "SEVERE_TOXICITY": {},
            "IDENTITY_ATTACK": {},
            "INSULT": {},
            "PROFANITY": {},
            "THREAT": {}
        },
        "languages": ["en"]
    }

    rate_limiter.acquire()

    try:
        response = requests.post(
            url,
            params={"key": api_key},
            headers={"Content-Type": "application/json"},
            json=data
        )

        scores = {}

        if response.status_code == 200:
            response_data = json.loads(response.text)
            attrscores = response_data['attributeScores']
            for attribute in attrscores:
                score = attrscores[attribute]['summaryScore']['value']
                scores[attribute.lower()] = score
        else:
            print("Error:", response.status_code, response.text)
    except requests.exceptions.RequestException as e:
        print("Error:", str(e))

    return scores

In [ ]:
get_perspective_scores("Example")

In [ ]:
# Use tqdm to display the progress bar
for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
    text = str(row['body'])
    scores = get_perspective_scores(text)
    
    # Add scores as new columns
    for attribute, score in scores.items():
        column_name = attribute
        df.loc[i, column_name] = score
    
        

In [ ]:
df.to_csv('comments_scores.csv', index=False)